### 데이터 인코딩

레이블 인코딩(label encoding)과 원-핫 인코딩(one-hot encoding)을 구분된다.

#### 레이블 인코딩 :  category feature를 코드형 숫자로 변환하는 것


books=['python','java','c']등이 있을때 그 각각의 값을 숫자로 변환하는 것 


In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

books=['python','java','c'] 

encoder=LabelEncoder()
encoder.fit(books)
encode_books=encoder.transform(books)
print(encode_books)
print(encoder.classes_)

[2 1 0]
['c' 'java' 'python']


#### 문제점 : 각 숫자로 변환을 하게 되므로 숫자의 크기로 가중치를 생각하게 됨

### 원-핫 인코딩(one-hot encoding)

각 feature 유형에 따라 새로운 feature를 추가해 고유값에 해당하는 컬럼에 1을 표시하고   
나머지는 0을 표시하는 방법

In [34]:
from sklearn.preprocessing import LabelEncoder
import  numpy as np
from sklearn.preprocessing import OneHotEncoder

books=['python','java','c'] 
encoder=LabelEncoder()

encoder.fit(books)
en_book=encoder.transform(books)

en_book=en_book[:, np.newaxis]
one_enc=OneHotEncoder()
one_enc.fit(en_book)
one_enbook=one_enc.transform(en_book).toarray()
print(one_enbook)


[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


### 평가 - 교차 검증(cross validation) 

train_test_split(): trainSet과 testSet으로 나누는 작업 

k-fold : 데이터를 비슷한 크기의 집합으로 k개 나누는 것

#### k-fold  cross_validation (k-겹 교차검증)
  그림에서처럼 k=5일때 데이터를 5개의 부분집합으로 분할 후   
  각 분할마다 하나의 폴드를 테스트용으로 사용, 4개 폴드를 훈련으로 사용한다.   
  이 과정을 반복하여 각 분할마다 정확도를 높인다.
   scikit-learn: cross_val_score
   
   
   =>shuffle 이용하면 fold 나누기 전 무작위로 섞음
  
<img src="./img/cross_validation.png" width="400px" height="450px" align="left">
<div style="clear:both; padding-top:10px">


####  stratified k-fold cross-validation(계층별 k-겹 교차검증)
 데이터가 편향되어 있을 경우(몰려있을 경우) => iris 경우 데이터가 편향되어 있음
    
    
<img src="./img/str_k_fold.png" width="550px" height="500px" align="left">
<div style="clear:both; padding-top:10px">
    
클래스 A가 90%, B가 10% 일 경우 계층별 교차 검증은 각 폴드에 90%와 10%이 되도록 만듬 
    
    
    


####  shuffle split cross validation(임의 분할 교차 검증)
    train_size 만큼의 포인트로 trainSet을 만들고, test_size 만큼으로 testSet을 만들도록 분할한다.
    => rain set과 test set의 크기를 유연하게 조절해야 할 때 유용하다.
    n_splits 횟수만큼 반복해서 분할 

   <img src="./img/shuffle_cross_validation.png" width="850px" height="700px" align="left">
   <div style="clear:both; padding-top:10px">
   샘플이 10개인 데이터셋을 5개의 포인트의 trainSet, 2개의 포인트의 testSet으로 4번 반복하여 나눔
    
    


#### k-fold cross_validation 

In [44]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
iris=load_iris()

scaler=StandardScaler()
iris_scaler=scaler.fit_transform(iris.data)

logreg=LogisticRegression()

scores=cross_val_score(logreg, iris_scaler, iris.target)
print(scores)

[0.96666667 1.         0.93333333 0.9        1.        ]


In [45]:
scores=cross_val_score(logreg, iris_scaler, iris.target, cv=7)
print(scores)

[0.95454545 1.         0.90909091 0.95238095 0.85714286 1.
 1.        ]


In [46]:
#교차정확도를 나타내기 위해 평균을 사용함

scores=cross_val_score(logreg, iris_scaler, iris.target, cv=7)
print(scores.mean())


0.9533085961657389


#### 문제점


In [73]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


iris=load_iris()
scaler=StandardScaler()
iris_scaler=scaler.fit_transform(iris.data)

logreg=LogisticRegression()
kfold=KFold(n_splits=5)
scores=cross_val_score(logreg, iris_scaler,iris.target ,cv=kfold)
print(scores)

[1.         1.         0.83333333 0.93333333 0.73333333]


In [74]:

logreg=LogisticRegression()
kfold=KFold(n_splits=3)
scores=cross_val_score(logreg, iris_scaler,iris.target ,cv=kfold)
print(scores)

[0. 0. 0.]


#### 해결책  -shufle을 이용하자

In [77]:
logreg=LogisticRegression()
kfold=KFold(n_splits=3,shuffle=True, random_state=0)
scores=cross_val_score(logreg, iris_scaler,iris.target ,cv=kfold)
print(scores)

[0.94 0.96 0.96]


#### LOOCV( Leave-One-Out-Cross-Validation)

폴드하나에 샘플 하나만 들어 있는 k-fold cross validation  
각 반복에서 하나의 데이터 포인트를 선택해 테스트 세트로 사용   
n개의 데이터 샘플에서 한개의 데이터 샘플을 test Set으로 하고, 나머지를 trainSet으로 검증하는 방법  
=> 장점 : 모든 샘플에 대해 다 한번씩 test하기때문에 많은 수의 training data를  
활용하여 model을 만들수 있다.   
=> 단점 :  많은 수의 model을 만들고 test해야 하기 때문에 computing time이  
굉장히 많이 걸린다. 

In [79]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

iris=load_iris()
scaler=StandardScaler()
iris_scaler=scaler.fit_transform(iris.data)
logreg=LogisticRegression()


loo=LeaveOneOut()
score=cross_val_score(logreg, iris_scaler, iris.target, cv=loo)

print(len(score))
print(score.mean())

150
0.9533333333333334


#### stratified k-fold cross-validation

In [86]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold


iris=load_iris()
scaler=StandardScaler()
iris_scaler=scaler.fit_transform(iris.data)
logreg=LogisticRegression()

stfold= StratifiedKFold(n_splits=3)
scores=cross_val_score(logreg, iris_scaler,iris.target ,cv=stfold)

print(scores)


[0.98 0.96 0.96]


#### shuffle split cross validation

In [96]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.model_selection import ShuffleSplit

iris=load_iris()
scaler=StandardScaler()
iris_scaler=scaler.fit_transform(iris.data)
logreg=LogisticRegression()

shuffle=ShuffleSplit(test_size=5, train_size=5, n_splits=10)
scores=cross_val_score(logreg, iris_scaler,iris.target ,cv=shuffle)

print(scores)

[1.  0.4 0.2 0.6 0.8 0.6 1.  0.4 0.6 1. ]
